In [1]:
%pip install datasets
%pip install evaluate
%pip install transformers
%pip install torch
%pip install numpy
%pip install peft

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

2024-11-19 13:08:34.255070: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <2D89372E-C77C-3351-8455-B2E540126864> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <A84DFEFF-287E-3B94-A7DB-731FA5F9CBBC> /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't 

In [ ]:
# !huggingface-cli login

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B")

In [ ]:
# # Step 1: Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import json

# Define the path to your JSONL file
file_path = '/content/drive/My Drive/Colab Notebooks/cs6158_project/data/resource_util.jsonl'

# Load the JSONL file
with open(file_path, 'r') as f:
    data = [json.loads(line) for line in f]

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Display the first few rows
print(df.head())


  non_functional_requirement  \
0                     memory   
1                     memory   
2                     memory   
3                     memory   
4                     memory   

                                              commit  \
0  https://github.com/aaronjwood/PortAuthority/co...   
1  https://github.com/aaronjwood/PortAuthority/co...   
2  https://github.com/aaronjwood/PortAuthority/co...   
3  https://github.com/aaronjwood/PortAuthority/co...   
4  https://github.com/vaginessa/TestBrowser-Light...   

                                  commit_message  \
0       '\\"Avoid potential resource leak\\n\\"'   
1                   '\\"Fix resource leak\\n\\"'   
2                     '\\"Fix memory leak\\n\\"'   
3  '\\"Fix resource leak in certain cases\\n\\"'   
4                 '\\"Fixed a memory leak\\n\\"'   

                                         source_code  \
0  package com.aaronjwood.portauthority.network;\...   
1  package com.aaronjwood.portauthority.runna

In [ ]:
!huggingface-cli login
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGr

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:

# Assuming df is already loaded as a pandas DataFrame
# Combine relevant columns (adjust as needed, here I'm using 'base_prompt' as input and 'target_code' as the output)
df['input_text'] = df['base_prompt'] + ' ' + df['coding_concepts'] + ' ' + df['chain_of_thought']
df['output_text'] = df['target_code']
# Set the pad token to be the eos token (common approach for models without a pad token)
tokenizer.pad_token = tokenizer.eos_token

# Now, you can proceed with tokenization
def tokenize_function(examples):
    return tokenizer(examples['input_text'], truncation=True, padding='max_length', max_length=512)

tokenized_train_data = train_data.map(tokenize_function, batched=True)

# Format the dataset for causal language modeling
tokenized_train_data = tokenized_train_data.map(
    lambda examples: {'labels': examples['input_ids']},
    batched=True
)

# Check tokenized data
print(tokenized_train_data[0])

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

{'input_text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nRewrite the given java program to optimize and improve the memory usage. Write the entire code and no other text in the response.\n```java\npackage com.aaronjwood.portauthority.network;\n\nimport android.app.Activity;\nimport android.database.Cursor;\n\nimport com.aaronjwood.portauthority.async.ScanPortsAsyncTask;\nimport com.aaronjwood.portauthority.db.Database;\nimport com.aaronjwood.portauthority.response.HostAsyncResponse;\n\npublic class Host {\n\n    /**\n     * Starts a port scan\n     *\n     * @param ip        IP address\n     * @param startPort The port to start scanning at\n     * @param stopPort  The port to stop scanning at\n     * @param delegate  Delegate to be called when the port scan has finished\n     */\n    public void scanPorts(String ip, int startPort, int stopPort, HostAsyncResponse delegate) {\n        new ScanPortsAsync

In [ ]:
# Define the LoRA configuration
lora_config = LoraConfig(
    r=8,  # rank for the LoRA layers
    lora_alpha=16,  # scaling factor for LoRA
    lora_dropout=0.1,  # dropout rate for LoRA
    task_type="CAUSAL_LM"  # causal language modeling task
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

In [ ]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./lora_model",
    evaluation_strategy="epoch",  # Evaluate every epoch
    learning_rate=2e-5,           # Learning rate
    per_device_train_batch_size=4,  # Batch size
    num_train_epochs=3,           # Number of epochs
    logging_dir='./logs',         # Where to save logs
    save_steps=500,               # Save checkpoints
    logging_steps=100,            # Log every 100 steps
    fp16=True                     # Enable mixed precision (if supported)
)

# Set up the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    tokenizer=tokenizer
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
